In [1]:
import json
import pandas as pd
import numpy as np
import modules.LP_Pool_State as lpps
import warnings
warnings.filterwarnings('ignore')
with open("config.json", 'r') as f:
    args = json.load(f)
with open("MP_ASSET_STATE.csv", "r") as outfile:
    MP_Asset_State = pd.read_csv(outfile, index_col=0)
with open("MP_ASSET.csv", "r") as outfile:
    MP_Asset = pd.read_csv(outfile, index_col=0)
LS_Opening = pd.read_csv("LS_Opening_p2", index_col=0)
LP_Deposit = pd.read_csv("LP_Deposit_p2", index_col=0)
LP_Withdraw = pd.read_csv("LP_Withdraw_p2", index_col=0)
SYS_LP_Withdraw = pd.read_csv("SYS_LP_Withdraw_p2", index_col=0)
PL_Interest = pd.read_csv("PL_Interest_p2",index_col=0)
LS_Repayment = pd.read_csv("LS_Repayment_p2", index_col=0)
LS_Liquidation = pd.read_csv("LS_Liquidation_p2", index_col=0)
LP_Pool_State = pd.read_csv("LP_Pool_State_p2", index_col=0)
PL_State = pd.read_csv("PL_State.csv",index_col=0)
LS_State = pd.read_csv("LS_State.csv",index_col=0)

TR_PROFIT(PER_TIMESTAMP) = args["LS_swap_prc"]*LS_Opening.loc[timestamp == timestamp,"LS_loan_amnt_stable"].sum()
+ PL_State["PL_LS_count_open"](timest)*(args["LS_tr_per_month"]/30)*args["tr_price"]+
PL_State["PL_LP_count_open"](timest)*(args["LP_tr_per_month"]/30)*args["tr_price"]+
args["treasury_interest"]#from LP_interest_calculate

args["LS_swap_prc"]
args["LS_tr_per_month"]
args["LP_tr_per_month"]
args["tr_price"]


#add initial record nolus tokens = 25000000

In [35]:
def TR_Profit_gen(LP_Pool_State,PL_State,LS_Opening,args):
    var = pd.DataFrame()
    var = LP_Pool_State[["LP_Pool_timestamp","SYS_TR_interest"]].groupby("LP_Pool_timestamp").sum().reset_index()
    var = var.rename(columns = {"LP_Pool_timestamp":"TR_Profit_timestamp"})
    swap_profit = LS_Opening[["LS_timestamp","LS_loan_amnt_stable"]].groupby("LS_timestamp").sum().reset_index()
    swap_profit["LS_loan_amnt_stable"] = swap_profit["LS_loan_amnt_stable"]*(args["LS_swap_prc"]/100)
    swap_profit = swap_profit.rename(columns={"LS_timestamp":"TR_Profit_timestamp"})
    ls_profit = PL_State[["PL_timestamp","PL_LS_count_open"]]
    ls_profit["ls_profit"] = ls_profit["PL_LS_count_open"]*(args["LS_tr_per_month"]/30)*args["tr_price"]
    ls_profit = ls_profit.rename(columns={"PL_timestamp":"TR_Profit_timestamp"})
    ls_profit = ls_profit.drop("PL_LS_count_open",axis=1)
    lp_profit = PL_State[["PL_timestamp","PL_LP_count_open"]]
    lp_profit["lp_profit"] = lp_profit["PL_LP_count_open"]*(args["LP_tr_per_month"]/30)*args["tr_price"]
    lp_profit = lp_profit.rename(columns={"PL_timestamp":"TR_Profit_timestamp"})
    lp_profit = lp_profit.drop("PL_LP_count_open",axis=1)


    var =  pd.merge(var,swap_profit, on=["TR_Profit_timestamp"], how="left")
    var =  pd.merge(var,ls_profit, on=["TR_Profit_timestamp"], how="left")
    var =  pd.merge(var,lp_profit, on=["TR_Profit_timestamp"], how="left")

    var["all_tr_profit"] = var["ls_profit"] + var["lp_profit"] + var["LS_loan_amnt_stable"] + var["SYS_TR_interest"]
    TR_Profit = pd.DataFrame({"TR_Profit_timestamp":var["TR_Profit_timestamp"],"TR_Profit_amnt_stable":var["all_tr_profit"]})
    
    lp_dep_h = [s for s in range(100,100+len(TR_Profit))]
    lp_dep_idx = lp_dep_h
    TR_Profit["TR_Profit_height"] = lp_dep_h
    TR_Profit["TR_Profit_idx"] = lp_dep_idx
    a =pd.DataFrame(args["symbol_digit"])
    c = a.loc[a["symbol"] == args["currency_stable"]]["digit"].values[0].astype(float)
    TR_Profit.loc[TR_Profit["TR_Profit_timestamp"] == min(TR_Profit["TR_Profit_timestamp"]),"TR_Profit_amnt_stable"] = args["nolus_token_count_ini"]*args["nolus_token_price_ini"]*10**c
    TR_Profit["TR_Profit_amnt_nls"] = args["nolus_token_count_ini"]
    
    return TR_Profit

In [36]:
prof = TR_Profit_gen(LP_Pool_State,PL_State,LS_Opening,args)

In [9]:
LS_Opening[["LS_timestamp","LS_loan_amnt_stable"]].groupby("LS_timestamp").sum().reset_index()

,LS_timestamp,LS_loan_amnt_stable
0,2020-08-22 03:00:00,17359204071938520
1,2020-08-23 03:00:00,94985908989817216
2,2020-08-24 03:00:00,108154539670877440
3,2020-08-25 03:00:00,171614747938897504
4,2020-08-26 03:00:00,208443855947964032
...,...,...
634,2022-08-17 03:00:00,1248612083676529920
635,2022-08-18 03:00:00,1065108827393392512
636,2022-08-19 03:00:00,1257384236752367872
637,2022-08-20 03:00:00,1168328653095717120


In [37]:
prof.round(0)#astype("uint64")

,TR_Profit_timestamp,TR_Profit_amnt_stable,TR_Profit_height,TR_Profit_idx,TR_Profit_amnt_nls
0,2020-08-21 03:00:00,2.500000e+18,100,100,25000000
1,2020-08-22 03:00:00,1.735920e+14,101,101,25000000
2,2020-08-23 03:00:00,9.498591e+14,102,102,25000000
3,2020-08-24 03:00:00,1.081545e+15,103,103,25000000
4,2020-08-25 03:00:00,1.716147e+15,104,104,25000000
...,...,...,...,...,...
726,2022-08-17 03:00:00,3.759928e+16,826,826,25000000
727,2022-08-18 03:00:00,3.692005e+16,827,827,25000000
728,2022-08-19 03:00:00,4.354330e+16,828,828,25000000
729,2022-08-20 03:00:00,4.385957e+16,829,829,25000000


In [2]:

def TR_Rewards_Distribution_ini():
    TR_Rewards_Distribution = pd.DataFrame({"TR_Rewards_height":[],"TR_Rewards_idx":[],"TR_Rewards_Pool_id":[],"TR_Rewards_timestamp":[],"TR_Rewards_amnt_stable":[],"TR_Rewards_amnt_nls":[]})
    return TR_Rewards_Distribution
rew = TR_Rewards_Distribution_ini()

In [ ]:
def TR_Rewards_Distribution_update(timestamp,i,TR_State,LP_Pool_State,args):
    rewards_dist = pd.read_csv(args["tvl_rewards_csv"])
    a = pd.DataFrame(args["symbol_digit"])
    c = a.loc[a["symbol"] == args["currency_stable"]]["digit"].values[0].astype(float)
    rewards_dist["pool_tvl"] = rewards_dist["pool_tvl"]*10**c
    pool_tvl = LP_Pool_State.loc[LP_Pool_State["LP_Pool_timestamp"]==timestamp,["LP_Pool_id","SYS_LP_Pool_TV_IntDep_stable"]]
    #df2['match'] = df2['Score1'].apply(lambda s: min(df1['Score1'].values, key=lambda x: abs(x - s)))
    pool_tvl[['match', 'pool_tvl']].merge(rewards_dist, how='left', left_on='match', right_on='SYS_LP_Pool_TV_IntDep_stable', suffixes=['', '_2'])
     .drop(columns=['match'])
    return pool


In [44]:
rewards_dist = pd.read_csv(args["tvl_rewards_csv"])
a = pd.DataFrame(args["symbol_digit"])
c = a.loc[a["symbol"] == args["currency_stable"]]["digit"].values[0].astype(float)
rewards_dist["pool_tvl"] = rewards_dist["pool_tvl"]*10**c
pool_tvl = LP_Pool_State.loc[LP_Pool_State["LP_Pool_timestamp"]=="2020-08-25 03:00:00",["LP_Pool_id","SYS_LP_Pool_TV_IntDep_stable"]]
temp = pool_tvl.rename(columns={"SYS_LP_Pool_TV_IntDep_stable":"pool_tvl"})
rewards_dist = pd.concat([rewards_dist,temp],axis=0)
rewards_dist = rewards_dist.sort_values(by="pool_tvl",ignore_index=True)
rewards_dist["nls_rewards"] = rewards_dist["nls_rewards"].interpolate(method="linear",limit_direction="both",axis=0)
rewards_dist = rewards_dist.drop(["pool_tvl"],axis=1)


,LP_Pool_id,SYS_LP_Pool_TV_IntDep_stable,nls_rewards
0,pid100,1.058252e+18,0.1
1,pid101,1.072279e+18,0.1


In [42]:
rewards_dist

,pool_tvl,nls_rewards,LP_Pool_id
0,1.058252e+18,0.100,pid100
1,1.072279e+18,0.100,pid101
2,1.000000e+19,0.100,NaN
3,1.500000e+19,0.100,NaN
4,2.000000e+19,0.100,NaN
5,2.500000e+19,0.095,NaN
6,3.000000e+19,0.095,NaN
7,3.500000e+19,0.095,NaN
8,4.000000e+19,0.090,NaN
9,4.500000e+19,0.090,NaN


In [26]:
pool_tvl["match"] = pool_tvl["SYS_LP_Pool_TV_IntDep_stable"].apply(lambda x: 

ValueError: Can only compare identically-labeled Series objects

In [ ]:
pool_tvl = pd.merge(pool_tvl,rewards_dist, how ="left", left_on="match")